In [1]:
# Data visualization
import numpy as np
import pandas as pd 
# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical

# Train-Test
from sklearn.model_selection import train_test_split
# Classification Report
from sklearn.metrics import classification_report, confusion_matrix

import pickle

import warnings
warnings.filterwarnings('ignore')

/Users/jainilpatel/PycharmProjects/Exercise-Correction/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## 1. Describe dataset and Train Model

### 1.1. Describe dataset

In [2]:
# Determine important landmarks for lunge
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
    "LEFT_HEEL",
    "RIGHT_HEEL",
    "LEFT_FOOT_INDEX",
    "RIGHT_FOOT_INDEX",
]

# Generate all columns of the data frame

HEADERS = ["label"] # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [4]:
def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


# Remove duplicate rows (optional)
def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''
    
    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_dataset.csv", sep=',', encoding='utf-8', index=False)


df = describe_dataset("./stage.train.csv")

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v', 'left_heel_x', 'left_heel_y', 'left_heel_z', 'left_heel_v', 'right_heel_x', 'right_heel_y', 'right_heel_z', 'right_heel_v', 'left_foot_index_x', 'left_foot_index_y', 'left_foot_index_z', 'left_foot_index_v', 'right_foot_index_x', 'right_foot_index_y', 'right_foot_index_z', 'right_foot_index_v']
Number of rows: 24244 
Number of columns: 53

Labels: 
D    8232
M    8148
I    7864
Name: label, dtype: int64

Missi

### 1.2. Preprocess data

In [5]:
# load dataset
df = pd.read_csv("./stage.train.csv")

# Categorizing label
df.loc[df["label"] == "I", "label"] = 0
df.loc[df["label"] == "M", "label"] = 1
df.loc[df["label"] == "D", "label"] = 2

print(f'Number of rows: {df.shape[0]} \nNumber of columns: {df.shape[1]}\n')
print(f"Labels: \n{df['label'].value_counts()}\n")

Number of rows: 24244 
Number of columns: 53

Labels: 
2    8232
1    8148
0    7864
Name: label, dtype: int64



In [6]:
# Standard Scaling of features
with open("./model/input_scaler.pkl", "rb") as f2:
    input_scaler = pickle.load(f2)

x = df.drop("label", axis = 1)
x = pd.DataFrame(input_scaler.transform(x))

y = df["label"]

# # Converting prediction to categorical
y_cat = to_categorical(y)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2)

### 1.3. Train model

In [8]:
model = Sequential()
model.add(Dense(52, input_dim = 52, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(52, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(14, activation = "relu"))
model.add(Dense(3, activation = "softmax"))
model.compile(Adam(lr = 0.01), "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 52)                2756      
                                                                 
 dropout (Dropout)           (None, 52)                0         
                                                                 
 dense_1 (Dense)             (None, 52)                2756      
                                                                 
 dropout_1 (Dropout)         (None, 52)                0         
                                                                 
 dense_2 (Dense)             (None, 14)                742       
                                                                 
 dense_3 (Dense)             (None, 3)                 45        
                                                                 
Total params: 6,299
Trainable params: 6,299
Non-trainabl

In [9]:
model.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test))

Epoch 1/100


2024-09-11 22:09:25.466853: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1940/1940 [==============================] - 1s 582us/step - loss: 0.1834 - accuracy: 0.9423 - val_loss: 0.0804 - val_accuracy: 0.9728
Epoch 2/100
1940/1940 [==============================] - 1s 540us/step - loss: 0.1156 - accuracy: 0.9704 - val_loss: 0.0573 - val_accuracy: 0.9860
Epoch 3/100
1940/1940 [==============================] - 1s 539us/step - loss: 0.1113 - accuracy: 0.9735 - val_loss: 0.0550 - val_accuracy: 0.9870
Epoch 4/100
1940/1940 [==============================] - 1s 535us/step - loss: 0.1016 - accuracy: 0.9752 - val_loss: 0.0563 - val_accuracy: 0.9897
Epoch 5/100
1940/1940 [==============================] - 1s 549us/step - loss: 0.1037 - accuracy: 0.9756 - val_loss: 0.0808 - val_accuracy: 0.9899
Epoch 6/100
1940/1940 [==============================] - 1s 544us/step - loss: 0.1191 - accuracy: 0.9760 - val_loss: 0.0768 - val_accuracy: 0.9887
Epoch 7/100
1940/1940 [==============================] - 1s 555us/step - loss: 0.0953 - accuracy: 0.9779 - val_loss: 0.0589 - val_

## 2. Model Evaluation

### 2.1. Train set evaluation

In [10]:
predict_x = model.predict(x_test) 
y_pred_class = np.argmax(predict_x, axis=1)

y_pred = model.predict(x_test)
y_test_class = np.argmax(y_test, axis=1)

confusion_matrix(y_test_class, y_pred_class)

152/152 [==============================] - 0s 230us/step


array([[1554,   11,    2],
       [   5, 1637,   13],
       [   0,    8, 1619]])

In [11]:
print(classification_report(y_test_class, y_pred_class))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      1567
           1       0.99      0.99      0.99      1655
           2       0.99      1.00      0.99      1627

    accuracy                           0.99      4849
   macro avg       0.99      0.99      0.99      4849
weighted avg       0.99      0.99      0.99      4849



### 2.2. Test set evaluation

In [13]:
test_df = pd.read_csv("./stage.test.csv")

# Categorizing label
test_df.loc[test_df["label"] == "I", "label"] = 0
test_df.loc[test_df["label"] == "M", "label"] = 1
test_df.loc[test_df["label"] == "D", "label"] = 2

In [14]:
# Standard Scaling of features
test_x = test_df.drop("label", axis = 1)
test_x = pd.DataFrame(input_scaler.transform(test_x))

test_y = test_df["label"]

# # Converting prediction to categorical
test_y_cat = to_categorical(test_y)

In [15]:
predict_x = model.predict(test_x) 
y_pred_class = np.argmax(predict_x, axis=1)
y_test_class = np.argmax(test_y_cat, axis=1)

confusion_matrix(y_test_class, y_pred_class)

38/38 [==============================] - 0s 330us/step


array([[400,   2,   0],
       [ 54, 316,  17],
       [  0,   2, 414]])

In [16]:
print(classification_report(y_test_class, y_pred_class))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93       402
           1       0.99      0.82      0.89       387
           2       0.96      1.00      0.98       416

    accuracy                           0.94      1205
   macro avg       0.94      0.94      0.94      1205
weighted avg       0.94      0.94      0.94      1205



## 3. Dump Model

In [17]:
# Dump the best model to a pickle file
with open("./model/lunge_model_deep_learning_jainil.pkl", "wb") as f:
    pickle.dump(model, f)

INFO:tensorflow:Assets written to: ram://3bb40aee-d034-4a78-b36b-3aafc661dd1c/assets
